In [40]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [41]:
url = 'https://en.wikipedia.org/wiki/List_of_mass_shootings_in_the_United_States_in_2025'

In [42]:
email = "wella2@byu.edu"
ua = f"STAT386-class-assignment/1.0 (+{email})"

r = requests.get(url, headers={"User-Agent": ua, "From": email}, timeout=15)
r.raise_for_status()

tables = pd.read_html(r.text)

# Sanity check: how many tables did we pull?
print(f"Total tables found: {len(tables)}")

# Select the second table (index 1)
try:
    main_tbl = tables[1]
except IndexError:
    raise RuntimeError("There are fewer than 2 tables on this page — cannot extract the second one.")

print(main_tbl.head())
print(f"Rows: {len(main_tbl)}")
print("Columns:", main_tbl.columns.tolist())

Total tables found: 5
    2025 date           Location    State or territory Dead Injured  Total  \
0  November 8  San Francisco (4)            California    0       5      5   
1  November 7         Wilmington              Delaware    0       5      5   
2  November 2    Los Angeles (7)            California    0       4      4   
3  November 2         Cincinnati                  Ohio    0       4      4   
4  November 2     Washington (4)  District of Columbia    0       5      5   

                                         Description  
0  A fight in the area of Outer Richmond led to a...  
1  Four men and a woman were shot in the Compton ...  
2  Four men were shot in the Winnetka neighborhoo...  
3  A dispute outside a nightclub in the Over-The-...  
4  Five people were shot in the Gateway neighborh...  
Rows: 385
Columns: ['2025 date', 'Location', 'State or territory', 'Dead', 'Injured', 'Total', 'Description']


/var/folders/f4/ftsj1j7n5837rlz4_q0rks500000gn/T/ipykernel_43409/4292007148.py:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(r.text)


In [43]:
# Rename columns for clarity
main_tbl.columns = [
    "Date",
    "City",
    "State or Territory",
    "Deaths",
    "Injuries",
    "Total Victims",
    "Description"
]
main_tbl.head()

,Date,City,State or Territory,Deaths,Injuries,Total Victims,Description
0,November 8,San Francisco (4),California,0,5,5,A fight in the area of Outer Richmond led to a...
1,November 7,Wilmington,Delaware,0,5,5,Four men and a woman were shot in the Compton ...
2,November 2,Los Angeles (7),California,0,4,4,Four men were shot in the Winnetka neighborhoo...
3,November 2,Cincinnati,Ohio,0,4,4,A dispute outside a nightclub in the Over-The-...
4,November 2,Washington (4),District of Columbia,0,5,5,Five people were shot in the Gateway neighborh...


In [44]:
# Rewrite date to be YYYY-MM-DD
main_tbl['Date'] = pd.to_datetime(
    main_tbl['Date'] + " 2025",
    format="%B %d %Y",
    errors='coerce'
)

main_tbl['Date'] = main_tbl['Date'].dt.strftime('%Y-%m-%d')
main_tbl.head()

,Date,City,State or Territory,Deaths,Injuries,Total Victims,Description
0,2025-11-08,San Francisco (4),California,0,5,5,A fight in the area of Outer Richmond led to a...
1,2025-11-07,Wilmington,Delaware,0,5,5,Four men and a woman were shot in the Compton ...
2,2025-11-02,Los Angeles (7),California,0,4,4,Four men were shot in the Winnetka neighborhoo...
3,2025-11-02,Cincinnati,Ohio,0,4,4,A dispute outside a nightclub in the Over-The-...
4,2025-11-02,Washington (4),District of Columbia,0,5,5,Five people were shot in the Gateway neighborh...


In [49]:
# Changing city entries from (City Name (number)) to just City Name and creating a new column for number
main_tbl["City"] = main_tbl["City"].apply(lambda x: re.sub(r" \(\d+\)", "", x))
city_freq = main_tbl["City"].value_counts()
main_tbl["Total City Frequency"] = main_tbl["City"].map(city_freq)

# Reorder columns
cols = main_tbl.columns.tolist()
cols.remove("City")
cols.remove("Total City Frequency")
cols.remove("Date")
new_order = ["Date", "City", "Total City Frequency"] + cols
df = main_tbl[new_order]

# Sort by frequency
df = df.sort_values(by="Date", ascending=True).reset_index(drop=True)

df.head()


,Date,City,Total City Frequency,State or Territory,Deaths,Injuries,Total Victims,Description
0,2025-01-01,Kankakee,1,Illinois,2,5,7,Two people were killed and five others were in...
1,2025-01-01,Dallas,7,Texas,1,3,4,An argument in the Wolf Creek neighborhood esc...
2,2025-01-01,New York City,7,New York,0,10,10,Four men opened fire on a queue of people outs...
3,2025-01-01,New Orleans,4,Louisiana,1[n 1],5,6,2025 New Orleans truck attack: A man who was i...
4,2025-01-03,Washington,4,District of Columbia,0,5[n 1],5,A dispute that escalated into a shooting wound...
